In [67]:
import numpy as np
from tqdm import tqdm
from time import sleep
import operator
import copy

In [68]:
class NN():
    """A class to build and train a neural network used to solve the XOR problem."""
    INPUT_NEURONS = 2
    OUTPUT_NEURONS = 1

    def __init__(self, hidden_neurons=2, pop_size=10):
        """Create a 3-layer NN with two input neurons, n hidden neurons and one output neuron.
        
        hidden_neurons -- the number of neurons in the NN's hidden layer
        pop_size       -- the bound for the number of individuals in a population
        """
        self.data = np.array([[0,0],
                              [0,1],
                              [1,0],
                              [1,1]])
        self.target = np.array([[0],[1],[1],[0]])

        self.h_neurons = hidden_neurons

        self.pop_size = pop_size
        
        self.w_hidden = [np.random.uniform(size=(NN.INPUT_NEURONS, self.h_neurons)) for _ in range(pop_size)]
        self.w_out = [np.random.uniform(size=(self.h_neurons, NN.OUTPUT_NEURONS)) for _ in range(pop_size)]

        self.b_hidden = [np.random.uniform(size=(1, self.h_neurons)) for _ in range(pop_size)]
        self.b_out = [np.random.uniform(size=(1, NN.OUTPUT_NEURONS)) for _ in range(pop_size)]

        self.z0s = []
        self.z1s = []

        print("Init.: ")
        self.print_weights()

    def ReLU(self, x):
        """ReLU activation function"""
        return x * (x > 0) 
    
    def sigmoid(self, x):
        """Sigmoid activation function"""
        return 1/(1 + np.exp(-x))
    
    def BCEL(self, y, y_hat):
        """Binary Cross-Entropy Loss
        
        y_hat -- the NN's prediction
        y     -- the ground truth
        """
        return -(y*np.log(y_hat) + (1-y) * np.log(1-y_hat))
    
    def execute(self, x):
        """Execute the NN for a given input, return its result."""
        result = self.sigmoid(self.forward_pass(0,x)[0])
        return np.round(result[0])
    
    def forward_pass(self, index, x):
        """Carry out the forward pass.
        
        index -- for using the weights of one specific individual of the population
        """
        x = np.array(x)
        z0 = np.dot(x, self.w_hidden[index]) + self.b_hidden[index]
        hidden = self.ReLU(z0)
        z1 = np.dot(hidden, self.w_out[index]) + self.b_out[index]
        return z1, z0


    def neuro_train(self, print_details : bool = True):
        """Train the NN's weights using evolutionary algorithms."""
        epoch = 1
        while self.accuracy(0)[0] != 1:
            
            if print_details: print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))

            copy_w_out = self.copy(self.w_out)
            copy_b_out = self.copy(self.b_out)
            copy_w_hidden = self.copy(self.w_hidden)
            copy_b_hidden = self.copy(self.b_hidden)
            
            self.mutation(copy_w_out)
            self.mutation(copy_b_out)
            self.mutation(copy_w_hidden)
            self.mutation(copy_b_hidden)
            
            self.w_out += copy_w_out
            self.w_hidden += copy_w_hidden
            self.b_out += copy_b_out
            self.b_hidden += copy_b_hidden

            f_list = self.fitness()
            self.selection(f_list, use_prop_slct=False)

            epoch += 1

        if print_details: print("Epoch: " + str(epoch) + " current acc: " + str(self.accuracy(0)[0]))
        self.print_weights()
    
    def copy(self, weight):
        """Create a copy of a given weight."""
        new_weight = []
        for w in weight:
            new_weight.append(w.copy())
        return new_weight
    
    def fitness(self):
        """Calculate the fitness of each individual in the population, using the respective loss."""
        f_list = []
        for i in range(len(self.w_out)-1):
            acc, loss = self.accuracy(i)
            f_list += [1-acc+loss]
        return f_list

    def selection(self, f_list, use_prop_slct : bool = False):
        """Select a subset of the population to be kept.
        
        f_list        -- a list of fitness values for every individual
        use_prop_slct -- whether to use fitness proportional selection (default False)
        """
        dic = dict()
        for i in range(len(f_list)):
            dic.update([(i,f_list[i])])
        
        sortedPop = sorted(dic.items(), key=operator.itemgetter(1), reverse=False)
        last = sortedPop[self.pop_size:]
        last.sort()
        c = 0
        for index in last:
            del self.w_out[index[0]-c]
            del self.w_hidden[index[0]-c]
            del self.b_out[index[0]-c]
            del self.b_hidden[index[0]-c]
            c += 1
        

    def crossover(self, weight):
        """Perform a crossover on a given weight."""

        w = np.zeros(shape=(len(weight), weight[0].shape[0], weight[0].shape[1]))
        lis = [i for i in range(len(weight))]
        np.random.shuffle(lis)
        for k in range(len(weight) // 2 - 1):
            for i in range(weight[0].shape[0]):
                for j in range(weight[0].shape[1]):
                    if np.random.randint(0,2):
                        w[lis[2*k]][i][j] = weight[lis[2*k+1]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k]][i][j]
                    else:
                        w[lis[2*k]][i][j] = weight[lis[2*k]][i][j]
                        w[lis[2*k+1]][i][j] = weight[lis[2*k+1]][i][j]
        return w

    def mutation(self, weight):
        """Mutate a given weight by adding values sampled from a normal destribution onto each entry."""
        for k in range(len(weight)):
            for i in range(len(weight[k])):
                for j in range(len(weight[k][i])):
                    weight[k][i][j] += np.random.normal(0,0.1)


    def print_weights(self):
        """Print the weights of the NN."""

        print("W_hidden: " + str(self.w_hidden[0])) 
        print("B_hidden: " + str(self.b_hidden[0])) 
        print("W_out: " + str(self.w_out[0])) 
        print("B_out: " + str(self.b_out[0]), end="\n\n")
        pass

    def accuracy(self, index):
        """Calculate the accuracy and loss of the NN for a specific individual.
        
        index -- for using the weights of one specific individual of the population
        """
        counter = 0
        for i in range(len(self.data)):
            if(self.execute(self.data[i]) == self.target[i]):
                counter += 1
        
        loss = 0
        for i in range(len(self.data)):
            loss += self.BCEL(self.target[i], self.sigmoid(self.forward_pass(index, self.data[i])[0]))

        return counter / len(self.data), loss / len(self.data)
    

In [69]:
nn = NN(4, pop_size=10)

Init.: 
W_hidden: [[0.17253823 0.16795174 0.37970585 0.63418161]
 [0.31538353 0.37999104 0.79131701 0.18273228]]
B_hidden: [[0.26139552 0.95255138 0.74896531 0.2601603 ]]
W_out: [[0.50589149]
 [0.33910437]
 [0.50106685]
 [0.21888637]]
B_out: [[0.69053099]]



In [70]:
nn.neuro_train(print_details=True)

Epoch: 1 current acc: 0.5
Epoch: 2 current acc: 0.5
Epoch: 3 current acc: 0.5
Epoch: 4 current acc: 0.5
Epoch: 5 current acc: 0.5
Epoch: 6 current acc: 0.5
Epoch: 7 current acc: 0.5
Epoch: 8 current acc: 0.5
Epoch: 9 current acc: 0.5
Epoch: 10 current acc: 0.5
Epoch: 11 current acc: 0.5
Epoch: 12 current acc: 0.5
Epoch: 13 current acc: 0.5
Epoch: 14 current acc: 0.5
Epoch: 15 current acc: 0.5
Epoch: 16 current acc: 0.5
Epoch: 17 current acc: 0.5
Epoch: 18 current acc: 0.5
Epoch: 19 current acc: 0.5
Epoch: 20 current acc: 0.5
Epoch: 21 current acc: 0.5
Epoch: 22 current acc: 0.5
Epoch: 23 current acc: 0.5
Epoch: 24 current acc: 0.5
Epoch: 25 current acc: 0.5
Epoch: 26 current acc: 0.5
Epoch: 27 current acc: 0.5
Epoch: 28 current acc: 0.5
Epoch: 29 current acc: 0.5
Epoch: 30 current acc: 0.75
Epoch: 31 current acc: 0.75
Epoch: 32 current acc: 0.5
Epoch: 33 current acc: 0.75
Epoch: 34 current acc: 0.75
Epoch: 35 current acc: 0.75
Epoch: 36 current acc: 1.0
W_hidden: [[ 0.57006516 -0.45327

In [46]:
nn.execute([0,1])

array([1.])

# Aufgabe 3


In [36]:
import tensorflow as tf

In [37]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2, 1)),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [38]:
loss_fn = tf.keras.losses.BinaryCrossentropy()

model.compile(loss=loss_fn,
              metrics=['accuracy'])

In [39]:
data = np.array([[0,0],
              [0,1],
              [1,0],
              [1,1]])

target = np.array([[0],[1],[1],[0]])

In [40]:
model.fit(data,target,epochs=200)

Epoch 1/200
1/1 [==============================] - 0s 411ms/step - loss: 0.7197 - accuracy: 0.5000
Epoch 2/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7171 - accuracy: 0.2500
Epoch 3/200
1/1 [==============================] - 0s 7ms/step - loss: 0.7152 - accuracy: 0.2500
Epoch 4/200
1/1 [==============================] - 0s 6ms/step - loss: 0.7137 - accuracy: 0.2500
Epoch 5/200
1/1 [==============================] - 0s 7ms/step - loss: 0.7123 - accuracy: 0.2500
Epoch 6/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7111 - accuracy: 0.2500
Epoch 7/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7100 - accuracy: 0.2500
Epoch 8/200
1/1 [==============================] - 0s 4ms/step - loss: 0.7089 - accuracy: 0.2500
Epoch 9/200
1/1 [==============================] - 0s 6ms/step - loss: 0.7079 - accuracy: 0.2500
Epoch 10/200
1/1 [==============================] - 0s 5ms/step - loss: 0.7070 - accuracy: 0.2500
Epoch 11/200
1/1 [=========

In [41]:
model.evaluate(data,  target, verbose=2)

1/1 - 0s - loss: 0.6182 - accuracy: 0.5000 - 95ms/epoch - 95ms/step


[0.6182018518447876, 0.5]